In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Instalar librerías

!pip install rdkit
!pip install Biopython
!pip install pandarallel
!pip install py3Dmol

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.3/34.3 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 29.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 4.2 MB/s eta 0:00:00
  Created wheel for pandarallel: filename=pandarallel-1.6.5-py3-none-any.whl size=16674 sha256=10248865c2ec27fa81eee59f44dd5b9b37a8935b517929729aa79ccf4094ebd7
  Stored in directory: /root/.cache/pip/wheels/b9/c6/5a/829298789e94348b81af52ab42c19d49da007306bbcc983827
Successfully built pandarallel


In [ ]:
#@title Abrir el dataframe

import os
import pandas as pd

input_folder = "/content/drive/MyDrive/TFM/T2/Ultrafast_Shape_Recognition"
input_file = os.path.join(input_folder, "df_final_familias.csv")
df_harm = pd.read_csv(input_file, sep = ',')
print(df_harm.shape[0],df_harm.columns)

40181 Index(['PDB_entry_id', 'Classification', 'Organism', 'Uniprot_id', 'Ligand_id',
       'Ligand_InChi', 'Experimental_method', 'Resolution',
       'Adding_Classification', 'Affinity', 'Coordenadas', 'Count_general',
       'Count_diff', 'Ligand_smiles'],
      dtype='object')


In [ ]:
#@title Expander el Dataframe por la columna "Coordenadas"

import pandas as pd
import ast

df_harm['Coordenadas'] = df_harm['Coordenadas'].apply(ast.literal_eval)

df_expandido = df_harm.explode('Coordenadas').reset_index(drop=True)
df_expandido.shape[0]

96717

El script está en ente enlace: https://gist.github.com/anku255/03dc35c5233a3fc59d60fdf62c3cda24

In [ ]:
#@title Función Ultrafast Shape Recognition

import math, os
from tqdm import tqdm

# --------- LECTOR PDB ---------
class Atom:
    def __init__(self, x, y, z):
        self.x = float(x)
        self.y = float(y)
        self.z = float(z)

    def getXCoordinate(self): return self.x
    def getYCoordinate(self): return self.y
    def getZCoordinate(self): return self.z

class Molecule:
    def __init__(self):
        self.atoms = []

    def addAtom(self, atom):
        self.atoms.append(atom)

    def getAtoms(self): return self.atoms
    def getAtom(self, i): return self.atoms[i]

def readPDBLigand(filepath):
    molecule = Molecule()
    with open(filepath, 'r') as f:
        for line in f:
            if line.startswith("HETATM") or line.startswith("ATOM"):
                try:
                    x = float(line[30:38])
                    y = float(line[38:46])
                    z = float(line[46:54])
                    molecule.addAtom(Atom(x, y, z))
                except:
                    continue
    return molecule

# --------- CÓDIGO USR ---------
class Point():
    def __init__(self, x=0, y=0, z=0):
        self.xCoordinate = x
        self.yCoordinate = y
        self.zCoordinate = z

class AtomIndexAndDistance():
    def __init__(self, index, distance):
        self.index = index
        self.distance = distance

class USR():
    def CalculateCentroid(self, aMolecule):
        centroid = Point()
        atoms = aMolecule.getAtoms()
        if not atoms:
            return centroid
        sx, sy, sz = 0.0, 0.0, 0.0
        for atom in atoms:
            sx += atom.getXCoordinate()
            sy += atom.getYCoordinate()
            sz += atom.getZCoordinate()
        n = len(atoms)
        centroid.xCoordinate = sx / n
        centroid.yCoordinate = sy / n
        centroid.zCoordinate = sz / n
        return centroid

    def EuclideanDistanceMeasure(self, aMolecule, point, i):
        atom = aMolecule.getAtom(i)
        dx = atom.getXCoordinate() - point.xCoordinate
        dy = atom.getYCoordinate() - point.yCoordinate
        dz = atom.getZCoordinate() - point.zCoordinate
        return math.sqrt(dx*dx + dy*dy + dz*dz)

    def ClosestAndFurthestAtomToCentroid(self, aMolecule):
        centroid = self.CalculateCentroid(aMolecule)
        distances = []
        for i in range(len(aMolecule.getAtoms())):
            dist = self.EuclideanDistanceMeasure(aMolecule, centroid, i)
            distances.append(AtomIndexAndDistance(i, dist))
        distances.sort(key=lambda x: x.distance)
        return [distances[0], distances[-1]]

    def FurthestAtomFromFurthestAtom(self, aMolecule, furthestAtom):
        index = furthestAtom.index
        target = aMolecule.getAtom(index)
        distances = []
        for i in range(len(aMolecule.getAtoms())):
            atom = aMolecule.getAtom(i)
            dx = atom.getXCoordinate() - target.getXCoordinate()
            dy = atom.getYCoordinate() - target.getYCoordinate()
            dz = atom.getZCoordinate() - target.getZCoordinate()
            dist = math.sqrt(dx*dx + dy*dy + dz*dz)
            distances.append(AtomIndexAndDistance(i, dist))
        distances.sort(key=lambda x: x.distance)
        return distances[-1].index

    def MomentToCentroid(self, momentId, aMolecule, centroid):
        atoms = aMolecule.getAtoms()
        distances = []
        for atom in atoms:
            dx = atom.getXCoordinate() - centroid.xCoordinate
            dy = atom.getYCoordinate() - centroid.yCoordinate
            dz = atom.getZCoordinate() - centroid.zCoordinate
            distances.append(math.sqrt(dx*dx + dy*dy + dz*dz))
        return self._moment(momentId, distances)

    def MomentToX(self, momentId, aMolecule, refIndex):
        ref = aMolecule.getAtom(refIndex)
        distances = []
        for atom in aMolecule.getAtoms():
            dx = atom.getXCoordinate() - ref.getXCoordinate()
            dy = atom.getYCoordinate() - ref.getYCoordinate()
            dz = atom.getZCoordinate() - ref.getZCoordinate()
            distances.append(math.sqrt(dx*dx + dy*dy + dz*dz))
        return self._moment(momentId, distances)

    def _moment(self, momentId, distances):
        n = len(distances)
        mean = sum(distances) / n
        if momentId == 1:
            return mean
        elif momentId == 2:
            return sum((d - mean)**2 for d in distances) / n
        elif momentId == 3:
            variance = sum((d - mean)**2 for d in distances) / n
            skewness = sum(abs(d - mean)**3 for d in distances) / n
            return skewness / (variance ** 1.5) if variance > 0 else 0.0

def getUSRDescriptor(aMolecule):
    usr = USR()
    centroid = usr.CalculateCentroid(aMolecule)
    v = usr.ClosestAndFurthestAtomToCentroid(aMolecule)
    ffaIndex = usr.FurthestAtomFromFurthestAtom(aMolecule, v[1])
    return [
        usr.MomentToCentroid(1, aMolecule, centroid),
        usr.MomentToCentroid(2, aMolecule, centroid),
        usr.MomentToCentroid(3, aMolecule, centroid),
        usr.MomentToX(1, aMolecule, v[0].index),
        usr.MomentToX(2, aMolecule, v[0].index),
        usr.MomentToX(3, aMolecule, v[0].index),
        usr.MomentToX(1, aMolecule, v[1].index),
        usr.MomentToX(2, aMolecule, v[1].index),
        usr.MomentToX(3, aMolecule, v[1].index),
        usr.MomentToX(1, aMolecule, ffaIndex),
        usr.MomentToX(2, aMolecule, ffaIndex),
        usr.MomentToX(3, aMolecule, ffaIndex),
    ]


In [ ]:
#@title Función USR para los ligandos agrupados

import os
import glob
from collections import defaultdict
from tqdm import tqdm
import math

#############################
# LECTOR PDB Y DESCRIPTORES #
#############################

# --------- LECTOR PDB ---------
class Atom:
    def __init__(self, x, y, z):
        self.x = float(x)
        self.y = float(y)
        self.z = float(z)

    def getXCoordinate(self):
        return self.x
    def getYCoordinate(self):
        return self.y
    def getZCoordinate(self):
        return self.z

class Molecule:
    def __init__(self):
        self.atoms = []

    def addAtom(self, atom):
        self.atoms.append(atom)

    def getAtoms(self):
        return self.atoms
    def getAtom(self, i):
        return self.atoms[i]

def readPDBLigand(filepath):
    molecule = Molecule()
    with open(filepath, 'r') as f:
        for line in f:
            if line.startswith("HETATM") or line.startswith("ATOM"):
                try:
                    x = float(line[30:38])
                    y = float(line[38:46])
                    z = float(line[46:54])
                    molecule.addAtom(Atom(x, y, z))
                except Exception as ex:
                    # Si ocurre algún error al convertir las coordenadas, se salta la línea.
                    continue
    return molecule

# --------- CÓDIGO USR ---------
class Point:
    def __init__(self, x=0, y=0, z=0):
        self.xCoordinate = x
        self.yCoordinate = y
        self.zCoordinate = z

class AtomIndexAndDistance:
    def __init__(self, index, distance):
        self.index = index
        self.distance = distance

class USR:
    def CalculateCentroid(self, aMolecule):
        centroid = Point()
        atoms = aMolecule.getAtoms()
        if not atoms:
            return centroid
        sx, sy, sz = 0.0, 0.0, 0.0
        for atom in atoms:
            sx += atom.getXCoordinate()
            sy += atom.getYCoordinate()
            sz += atom.getZCoordinate()
        n = len(atoms)
        centroid.xCoordinate = sx / n
        centroid.yCoordinate = sy / n
        centroid.zCoordinate = sz / n
        return centroid

    def EuclideanDistanceMeasure(self, aMolecule, point, i):
        atom = aMolecule.getAtom(i)
        dx = atom.getXCoordinate() - point.xCoordinate
        dy = atom.getYCoordinate() - point.yCoordinate
        dz = atom.getZCoordinate() - point.zCoordinate
        return math.sqrt(dx*dx + dy*dy + dz*dz)

    def ClosestAndFurthestAtomToCentroid(self, aMolecule):
        centroid = self.CalculateCentroid(aMolecule)
        distances = []
        for i in range(len(aMolecule.getAtoms())):
            dist = self.EuclideanDistanceMeasure(aMolecule, centroid, i)
            distances.append(AtomIndexAndDistance(i, dist))
        distances.sort(key=lambda x: x.distance)
        return [distances[0], distances[-1]]

    def FurthestAtomFromFurthestAtom(self, aMolecule, furthestAtom):
        index = furthestAtom.index
        target = aMolecule.getAtom(index)
        distances = []
        for i in range(len(aMolecule.getAtoms())):
            atom = aMolecule.getAtom(i)
            dx = atom.getXCoordinate() - target.getXCoordinate()
            dy = atom.getYCoordinate() - target.getYCoordinate()
            dz = atom.getZCoordinate() - target.getZCoordinate()
            dist = math.sqrt(dx*dx + dy*dy + dz*dz)
            distances.append(AtomIndexAndDistance(i, dist))
        distances.sort(key=lambda x: x.distance)
        return distances[-1].index

    def MomentToCentroid(self, momentId, aMolecule, centroid):
        atoms = aMolecule.getAtoms()
        distances = []
        for atom in atoms:
            dx = atom.getXCoordinate() - centroid.xCoordinate
            dy = atom.getYCoordinate() - centroid.yCoordinate
            dz = atom.getZCoordinate() - centroid.zCoordinate
            distances.append(math.sqrt(dx*dx + dy*dy + dz*dz))
        return self._moment(momentId, distances)

    def MomentToX(self, momentId, aMolecule, refIndex):
        ref = aMolecule.getAtom(refIndex)
        distances = []
        for atom in aMolecule.getAtoms():
            dx = atom.getXCoordinate() - ref.getXCoordinate()
            dy = atom.getYCoordinate() - ref.getYCoordinate()
            dz = atom.getZCoordinate() - ref.getZCoordinate()
            distances.append(math.sqrt(dx*dx + dy*dy + dz*dz))
        return self._moment(momentId, distances)

    def _moment(self, momentId, distances):
        n = len(distances)
        mean = sum(distances) / n
        if momentId == 1:
            return mean
        elif momentId == 2:
            return sum((d - mean)**2 for d in distances) / n
        elif momentId == 3:
            variance = sum((d - mean)**2 for d in distances) / n
            skewness = sum(abs(d - mean)**3 for d in distances) / n
            return skewness / (variance ** 1.5) if variance > 0 else 0.0

def getUSRDescriptor(aMolecule):
    usr = USR()
    centroid = usr.CalculateCentroid(aMolecule)
    v = usr.ClosestAndFurthestAtomToCentroid(aMolecule)
    ffaIndex = usr.FurthestAtomFromFurthestAtom(aMolecule, v[1])
    return [
        usr.MomentToCentroid(1, aMolecule, centroid),
        usr.MomentToCentroid(2, aMolecule, centroid),
        usr.MomentToCentroid(3, aMolecule, centroid),
        usr.MomentToX(1, aMolecule, v[0].index),
        usr.MomentToX(2, aMolecule, v[0].index),
        usr.MomentToX(3, aMolecule, v[0].index),
        usr.MomentToX(1, aMolecule, v[1].index),
        usr.MomentToX(2, aMolecule, v[1].index),
        usr.MomentToX(3, aMolecule, v[1].index),
        usr.MomentToX(1, aMolecule, ffaIndex),
        usr.MomentToX(2, aMolecule, ffaIndex),
        usr.MomentToX(3, aMolecule, ffaIndex),
    ]

#################################
# AGRUPAMIENTO Y PROCESAMIENTO  #
#################################

# Ruta de la carpeta de entrada con archivos PDB
input_folder = "/content/drive/MyDrive/TFM/T2/Ultrafast_Shape_Recognition/ligandos_pdb"

# Ruta de salida para los archivos CSV de USR
output_folder = "/content/drive/MyDrive/TFM/T2/Ultrafast_Shape_Recognition/Output_USR_1"
os.makedirs(output_folder, exist_ok=True)

# 1. Agrupar por el nombre del ligando
ligandos_por_nombre = defaultdict(list)
pdb_files = sorted(glob.glob(os.path.join(input_folder, "*.pdb")))

for filepath in pdb_files:
    base = os.path.basename(filepath)
    parts = base.split("_")
    if len(parts) < 2:
        # Si no se puede dividir correctamente, se ignora
        continue
    ligand_name = parts[1]
    ligandos_por_nombre[ligand_name].append(filepath)

# 2. Procesar cada grupo y generar un CSV para cada ligando
for ligand_name, archivos in ligandos_por_nombre.items():
    output_path = os.path.join(output_folder, f"USR_{ligand_name}.csv")
    with open(output_path, 'w') as out:
        out.write("conformation," + ",".join([f"USR_{i+1}" for i in range(12)]) + "\n")
        for pdb_file in tqdm(archivos, desc=f"Procesando {ligand_name}"):
            try:
                mol = readPDBLigand(pdb_file)
                descriptor = getUSRDescriptor(mol)
                desc_line = ",".join([f"{d:.5f}" for d in descriptor])
                out.write(f"{os.path.basename(pdb_file)},{desc_line}\n")
            except Exception as e:
                print(f"Error en {pdb_file}: {e}")


Procesando BCL:  10%|▉         | 123/1263 [01:22<12:13,  1.55it/s]

Error en /content/drive/MyDrive/TFM/T2/Ultrafast_Shape_Recognition/ligandos_pdb/1QOV_BCL_L_1303_ligando.pdb: list index out of range


Procesando CB3:  37%|███▋      | 11/30 [00:07<00:12,  1.57it/s]

Error en /content/drive/MyDrive/TFM/T2/Ultrafast_Shape_Recognition/ligandos_pdb/1QZF_CB3_D_616_ligando.pdb: list index out of range


Procesando PLM:   2%|▎         | 18/720 [00:13<07:41,  1.52it/s]

...

Procesando U:  97%|█████████▋| 29/30 [00:20<00:00,  1.44it/s]

Error en /content/drive/MyDrive/TFM/T2/Ultrafast_Shape_Recognition/ligandos_pdb/9BF2_U_B_601_ligando.pdb: list index out of range


Procesando U: 100%|██████████| 30/30 [00:20<00:00,  1.44it/s]


Error en /content/drive/MyDrive/TFM/T2/Ultrafast_Shape_Recognition/ligandos_pdb/9BF2_U_C_601_ligando.pdb: list index out of range


Procesando DSN:   8%|▊         | 2/24 [00:01<00:15,  1.39it/s]

## Segundo intento

Este código salta los archivos ya analizados para evitar repetir la investigación.

In [ ]:
import os
import glob
from collections import defaultdict
from tqdm import tqdm
import math

#############################
# LECTOR PDB Y DESCRIPTORES #
#############################

# --------- LECTOR PDB ---------
class Atom:
    def __init__(self, x, y, z):
        self.x = float(x)
        self.y = float(y)
        self.z = float(z)

    def getXCoordinate(self):
        return self.x
    def getYCoordinate(self):
        return self.y
    def getZCoordinate(self):
        return self.z

class Molecule:
    def __init__(self):
        self.atoms = []

    def addAtom(self, atom):
        self.atoms.append(atom)

    def getAtoms(self):
        return self.atoms
    def getAtom(self, i):
        return self.atoms[i]

def readPDBLigand(filepath):
    molecule = Molecule()
    with open(filepath, 'r') as f:
        for line in f:
            if line.startswith("HETATM") or line.startswith("ATOM"):
                try:
                    x = float(line[30:38])
                    y = float(line[38:46])
                    z = float(line[46:54])
                    molecule.addAtom(Atom(x, y, z))
                except Exception as ex:
                    # Si ocurre algún error al convertir las coordenadas, se salta la línea.
                    continue
    return molecule

# --------- CÓDIGO USR ---------
class Point:
    def __init__(self, x=0, y=0, z=0):
        self.xCoordinate = x
        self.yCoordinate = y
        self.zCoordinate = z

class AtomIndexAndDistance:
    def __init__(self, index, distance):
        self.index = index
        self.distance = distance

class USR:
    def CalculateCentroid(self, aMolecule):
        centroid = Point()
        atoms = aMolecule.getAtoms()
        if not atoms:
            return centroid
        sx, sy, sz = 0.0, 0.0, 0.0
        for atom in atoms:
            sx += atom.getXCoordinate()
            sy += atom.getYCoordinate()
            sz += atom.getZCoordinate()
        n = len(atoms)
        centroid.xCoordinate = sx / n
        centroid.yCoordinate = sy / n
        centroid.zCoordinate = sz / n
        return centroid

    def EuclideanDistanceMeasure(self, aMolecule, point, i):
        atom = aMolecule.getAtom(i)
        dx = atom.getXCoordinate() - point.xCoordinate
        dy = atom.getYCoordinate() - point.yCoordinate
        dz = atom.getZCoordinate() - point.zCoordinate
        return math.sqrt(dx*dx + dy*dy + dz*dz)

    def ClosestAndFurthestAtomToCentroid(self, aMolecule):
        centroid = self.CalculateCentroid(aMolecule)
        distances = []
        for i in range(len(aMolecule.getAtoms())):
            dist = self.EuclideanDistanceMeasure(aMolecule, centroid, i)
            distances.append(AtomIndexAndDistance(i, dist))
        distances.sort(key=lambda x: x.distance)
        return [distances[0], distances[-1]]

    def FurthestAtomFromFurthestAtom(self, aMolecule, furthestAtom):
        index = furthestAtom.index
        target = aMolecule.getAtom(index)
        distances = []
        for i in range(len(aMolecule.getAtoms())):
            atom = aMolecule.getAtom(i)
            dx = atom.getXCoordinate() - target.getXCoordinate()
            dy = atom.getYCoordinate() - target.getYCoordinate()
            dz = atom.getZCoordinate() - target.getZCoordinate()
            dist = math.sqrt(dx*dx + dy*dy + dz*dz)
            distances.append(AtomIndexAndDistance(i, dist))
        distances.sort(key=lambda x: x.distance)
        return distances[-1].index

    def MomentToCentroid(self, momentId, aMolecule, centroid):
        atoms = aMolecule.getAtoms()
        distances = []
        for atom in atoms:
            dx = atom.getXCoordinate() - centroid.xCoordinate
            dy = atom.getYCoordinate() - centroid.yCoordinate
            dz = atom.getZCoordinate() - centroid.zCoordinate
            distances.append(math.sqrt(dx*dx + dy*dy + dz*dz))
        return self._moment(momentId, distances)

    def MomentToX(self, momentId, aMolecule, refIndex):
        ref = aMolecule.getAtom(refIndex)
        distances = []
        for atom in aMolecule.getAtoms():
            dx = atom.getXCoordinate() - ref.getXCoordinate()
            dy = atom.getYCoordinate() - ref.getYCoordinate()
            dz = atom.getZCoordinate() - ref.getZCoordinate()
            distances.append(math.sqrt(dx*dx + dy*dy + dz*dz))
        return self._moment(momentId, distances)

    def _moment(self, momentId, distances):
        n = len(distances)
        mean = sum(distances) / n
        if momentId == 1:
            return mean
        elif momentId == 2:
            return sum((d - mean)**2 for d in distances) / n
        elif momentId == 3:
            variance = sum((d - mean)**2 for d in distances) / n
            skewness = sum(abs(d - mean)**3 for d in distances) / n
            return skewness / (variance ** 1.5) if variance > 0 else 0.0

def getUSRDescriptor(aMolecule):
    usr = USR()
    centroid = usr.CalculateCentroid(aMolecule)
    v = usr.ClosestAndFurthestAtomToCentroid(aMolecule)
    ffaIndex = usr.FurthestAtomFromFurthestAtom(aMolecule, v[1])
    return [
        usr.MomentToCentroid(1, aMolecule, centroid),
        usr.MomentToCentroid(2, aMolecule, centroid),
        usr.MomentToCentroid(3, aMolecule, centroid),
        usr.MomentToX(1, aMolecule, v[0].index),
        usr.MomentToX(2, aMolecule, v[0].index),
        usr.MomentToX(3, aMolecule, v[0].index),
        usr.MomentToX(1, aMolecule, v[1].index),
        usr.MomentToX(2, aMolecule, v[1].index),
        usr.MomentToX(3, aMolecule, v[1].index),
        usr.MomentToX(1, aMolecule, ffaIndex),
        usr.MomentToX(2, aMolecule, ffaIndex),
        usr.MomentToX(3, aMolecule, ffaIndex),
    ]

In [ ]:
import os
import glob
from collections import defaultdict
from tqdm import tqdm

#################################
# AGRUPAMIENTO Y PROCESAMIENTO  #
#################################

# Ruta de la carpeta de entrada con archivos PDB
input_folder = "/content/drive/MyDrive/TFM/T2/Ultrafast_Shape_Recognition/ligandos_pdb"

# Ruta de salida para los archivos CSV de USR
output_folder = "/content/drive/MyDrive/TFM/T2/Ultrafast_Shape_Recognition/Output_USR"
os.makedirs(output_folder, exist_ok=True)

# 1. Agrupar por el nombre del ligando, que se encuentra en el índice [1]
ligandos_por_nombre = defaultdict(list)
pdb_files = sorted(glob.glob(os.path.join(input_folder, "*.pdb")))

for filepath in pdb_files:
    base = os.path.basename(filepath)
    parts = base.split("_")
    if len(parts) < 2:
        # Si no se puede dividir correctamente, se ignora
        continue
    ligand_name = parts[1]  # Ejemplo: "GNP"
    ligandos_por_nombre[ligand_name].append(filepath)

# 2. Procesar cada grupo y generar un CSV para cada ligando, omitiendo si ya existe
for ligand_name, archivos in ligandos_por_nombre.items():
    output_path = os.path.join(output_folder, f"USR_{ligand_name}.csv")

    # Si el CSV para este ligando ya existe, se salta el procesamiento
    if os.path.exists(output_path):
        print(f"El archivo {output_path} ya existe. Saltando {ligand_name}.")
        continue

    with open(output_path, 'w') as out:
        out.write("conformation," + ",".join([f"USR_{i+1}" for i in range(12)]) + "\n")
        for pdb_file in tqdm(archivos, desc=f"Procesando {ligand_name}"):
            try:
                mol = readPDBLigand(pdb_file)
                descriptor = getUSRDescriptor(mol)
                desc_line = ",".join([f"{d:.5f}" for d in descriptor])
                out.write(f"{os.path.basename(pdb_file)},{desc_line}\n")
            except Exception as e:
                print(f"Error en {pdb_file}: {e}")


El archivo /content/drive/MyDrive/TFM/T2/Ultrafast_Shape_Recognition/Output_USR/USR_NBN.csv ya existe. Saltando NBN.
El archivo /content/drive/MyDrive/TFM/T2/Ultrafast_Shape_Recognition/Output_USR/USR_GCP.csv ya existe. Saltando GCP.
El archivo /content/drive/MyDrive/TFM/T2/Ultrafast_Shape_Recognition/Output_USR/USR_AMP.csv ya existe. Saltando AMP.
El archivo /content/drive/MyDrive/TFM/T2/Ultrafast_Shape_Recognition/Output_USR/USR_SAS.csv ya existe. Saltando SAS.
El archivo /content/drive/MyDrive/TFM/T2/Ultrafast_Shape_Recognition/Output_USR/USR_3PG.csv ya existe. Saltando 3PG.
El archivo /content/drive/MyDrive/TFM/T2/Ultrafast_Shape_Recognition/Output_USR/USR_GTX.csv ya existe. Saltando GTX.
El archivo /content/drive/MyDrive/TFM/T2/Ultrafast_Shape_Recognition/Output_USR/USR_BNZ.csv ya existe. Saltando BNZ.
El archivo /content/drive/MyDrive/TFM/T2/Ultrafast_Shape_Recognition/Output_USR/USR_HED.csv ya existe. Saltando HED.
El archivo /content/drive/MyDrive/TFM/T2/Ultrafast_Shape_Recogni